In [1]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=fad26ad06c142d3f8bf78a6b12e266315af2f36e0fd4a914282ddbe5ff5261a9
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [2]:
import wikipedia
import os
import time
import random

# 📁 Create output directory
corpus_dir = "wiki_corpus"
os.makedirs(corpus_dir, exist_ok=True)

# ⚙️ Settings
NUM_DOCS = 100
MIN_WORDS = 50
collected = 0
titles_seen = set()

print("🔍 Gathering documents...")

while collected < NUM_DOCS:
    try:
        # Get a random Wikipedia title
        title = wikipedia.random()
        if title in titles_seen:
            continue
        titles_seen.add(title)

        # Try to fetch the full page
        page = wikipedia.page(title)
        content = page.content.strip()

        # Check word count
        if len(content.split()) >= MIN_WORDS:
            filename = f"{corpus_dir}/doc_{collected+1}_{title.replace('/', '_')[:50]}.txt"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(content)
            collected += 1
            print(f"✅ {collected}: {title}")

        time.sleep(0.5)  # polite delay

    except (wikipedia.exceptions.DisambiguationError, wikipedia.exceptions.PageError):
        continue
    except Exception as e:
        print(f"⚠️ Error: {e}")
        time.sleep(1)
        continue

print(f"\n🎉 Done! {collected} documents saved in the '{corpus_dir}' directory.")


🔍 Gathering documents...
✅ 1: Samrat Choudhary
✅ 2: Petersbach
✅ 3: Denivia
✅ 4: Yanagushevo
✅ 5: John Chirkun
✅ 6: New York State Route 31A
✅ 7: Ti Manno
✅ 8: Deception (video game series)
✅ 9: Personal care products
✅ 10: Jaugilka
✅ 11: Star Empires (play-by-mail game)
✅ 12: Darren Haydar
✅ 13: Harris Academy Chafford Hundred
✅ 14: Nurul Islam (lawyer)
✅ 15: Geoff Workman
✅ 16: 1978 Marshall Thundering Herd football team


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


✅ 17: Become Desert
✅ 18: Phil DeLara
✅ 19: 2013 Central American Junior and Youth Championships in Athletics
✅ 20: John Templeton-Cotill
✅ 21: Lindsay/Kawartha Lakes Municipal Airport
✅ 22: 1849 in Chile
✅ 23: Daniela Vallega-Neu
✅ 24: Thomas Bryan (Irish republican)
✅ 25: WTAL
✅ 26: Tork-e Sharqi Rural District
✅ 27: Anne de Bruin
✅ 28: JTE Multimedia
✅ 29: Dumb Head
✅ 30: San Jose Hospital & Trauma Center
✅ 31: Culture Clash (album)
✅ 32: Chance medley
✅ 33: Brandon West
✅ 34: Limburgs Symfonie Orkest
✅ 35: AhlulBayt News Agency
✅ 36: National Pasta Association
✅ 37: Sullivan Walker
✅ 38: Bicyrtes viduatus
✅ 39: Datnioides undecimradiatus
✅ 40: James McGrath Morris
✅ 41: Grissom
✅ 42: Keria
✅ 43: Rodica Doehnert
✅ 44: Darcia Narvaez
✅ 45: No. 170 Squadron RAF
✅ 46: Weightlifting at the 2004 Summer Olympics – Men's 94 kg
✅ 47: Venturi Transport Protocol
✅ 48: Mark Krikorian (activist)
✅ 49: Table Tennis World Masters
✅ 50: RubyMotion
✅ 51: Wexford Collegiate School for the Arts
✅ 52:

In [ ]:
import os
import re
import math
from collections import defaultdict, Counter

# 📁 Load documents from corpus directory
corpus_dir = "wiki_corpus"
documents = {}

print("📂 Loading documents...")
for filename in os.listdir(corpus_dir):
    filepath = os.path.join(corpus_dir, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        documents[filename] = f.read()

# 🧹 Preprocessing function
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.split()

# 🔍 Build Inverted Index
inverted_index = defaultdict(list)
doc_lengths = {}
total_docs = len(documents)

df = Counter()
print("🔧 Building inverted index...")

for doc_id, content in documents.items():
    words = preprocess(content)
    doc_lengths[doc_id] = len(words)
    counted = Counter(words)
    for word, freq in counted.items():
        inverted_index[word].append((doc_id, freq))
        df[word] += 1

# 📈 Compute TF-IDF
def compute_scores(query):
    query_words = preprocess(query)
    scores = defaultdict(float)

    for word in query_words:
        if word in inverted_index:
            idf = math.log(total_docs / (1 + df[word]))
            for doc_id, freq in inverted_index[word]:
                tf = freq / doc_lengths[doc_id]
                scores[doc_id] += tf * idf

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# ✂️ Snippet extractor
def extract_snippet(doc_text, query):
    query_words = set(preprocess(query))
    words = preprocess(doc_text)
    for i, word in enumerate(words):
        if word in query_words:
            start = max(i - 5, 0)
            end = min(i + 5, len(words))
            return "... " + " ".join(words[start:end]) + " ..."
    return "..."

# 🔎 Search Interface
def search():
    print("\n🔎 Welcome to the Wikipedia Corpus Search!")
    while True:
        query = input("\nEnter your search query (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break

        results = compute_scores(query)
        if not results:
            print("\n❌ No results found.")
        else:
            print(f"\n✅ Top {min(5, len(results))} results:")
            for doc_id, score in results[:5]:
                snippet = extract_snippet(documents[doc_id], query)
                print(f"\n📄 {doc_id} (Score: {score:.4f})\n{snippet}")

# 🚀 Run search engine
if __name__ == "__main__":
    search()


📂 Loading documents...
🔧 Building inverted index...

🔎 Welcome to the Wikipedia Corpus Search!

✅ Top 1 results:

📄 doc_100_Jim Parker (rugby union).txt (Score: 0.0981)
... james hislop parker 1 february 1897 11 ...

❌ No results found.

❌ No results found.

❌ No results found.

✅ Top 5 results:

📄 doc_38_1785 South Carolina gubernatorial election.txt (Score: 0.0480)
... lieutenant governor of south carolina william moultrie was elected by ...

📄 doc_69_Fred Fry.txt (Score: 0.0315)
... rabbitohs footballers ed fry and william fry references the encyclopedia ...

📄 doc_74_William Pearson (priest).txt (Score: 0.0179)
... the venerable william pearson lld 10 august ...

📄 doc_40_William Adams (cricketer, born 1885).txt (Score: 0.0109)
... william adams 17 april 1885 ...

📄 doc_47_Monte Walsh (2003 film).txt (Score: 0.0027)
... the ranch of cal brennan william devane where they meet ...

✅ Top 2 results:

📄 doc_47_Monte Walsh (2003 film).txt (Score: 0.1202)
... monte walsh is a 2003 ...

📄

In [3]:
!zip -r corups.zip /content/wiki_corpus

  adding: content/wiki_corpus/ (stored 0%)
  adding: content/wiki_corpus/doc_4_Yanagushevo.txt (deflated 30%)
  adding: content/wiki_corpus/doc_57_African Morning Post.txt (deflated 34%)
  adding: content/wiki_corpus/doc_47_Venturi Transport Protocol.txt (deflated 45%)
  adding: content/wiki_corpus/doc_42_Keria.txt (deflated 60%)
  adding: content/wiki_corpus/doc_55_Jean Antoine Joseph Fauchet.txt (deflated 45%)
  adding: content/wiki_corpus/doc_26_Tork-e Sharqi Rural District.txt (deflated 46%)
  adding: content/wiki_corpus/doc_67_Legal Immigration Family Equity Act.txt (deflated 64%)
  adding: content/wiki_corpus/doc_98_Always Summer.txt (deflated 40%)
  adding: content/wiki_corpus/doc_52_Airline Highway.txt (deflated 57%)
  adding: content/wiki_corpus/doc_11_Star Empires (play-by-mail game).txt (deflated 62%)
  adding: content/wiki_corpus/doc_20_John Templeton-Cotill.txt (deflated 41%)
  adding: content/wiki_corpus/doc_91_2023 Arlington Renegades season.txt (deflated 58%)
  adding: 